In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  
from __future__ import absolute_import

In [2]:
from extract_data import *
from finbeat_calc import *
from plot_track import *
from plot_accel import *
from check_plots import *
from plot_analysis import *
from make_subset import *
from read_data import *
from fourier_analysis import *
import pywt
import pylab
import numpy as np
import pickle

tracklist = extract_data()

# Calculate finbeat peaks and troughs
# Extract finbeat periods, amplitudes, and subsequent accel
finbeats, finbeat_byP, finbeat_byT = finbeat_calc(tracklist.keys(),
                                             tracklist)

C:\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:595: RankWarning: Polyfit may be poorly conditioned
  warnings.warn(msg, RankWarning)
C:\Anaconda3\

In [3]:
bass_subset = make_subset(group_by1='species', identifier1='Bass', tracklist=tracklist)
bass_burst_subset = make_subset(group_by1 = 'species', identifier1='Bass', 
                                 tracklist=tracklist, group_by2 = 'behavior', identifier2='B')
bass_steady_subset = make_subset(group_by1 = 'species', identifier1='Bass', 
                                 tracklist=tracklist, group_by2 = 'behavior', identifier2='S')
bass_linacc_subset = make_subset(group_by1 = 'species', identifier1='Bass', 
                                 tracklist=tracklist, group_by2 = 'behavior', identifier2='A')

trout_subset = make_subset(group_by1 = 'species', identifier1 ='BTrout', tracklist=tracklist)
trout_burst_subset = make_subset(group_by1 = 'species', identifier1='BTrout', 
                                 tracklist=tracklist, group_by2 = 'behavior', identifier2='B')
trout_steady_subset = make_subset(group_by1 = 'species', identifier1='BTrout', 
                                 tracklist=tracklist, group_by2 = 'behavior', identifier2='S')
trout_linacc_subset = make_subset(group_by1 = 'species', identifier1='BTrout', 
                                 tracklist=tracklist, group_by2 = 'behavior', identifier2='A')

In [45]:
print(trout_burst_subset)

['BTrout2S04', 'BTrout2S10', 'BTrout2S07', 'BTrout2S05', 'BTrout1S11', 'BTrout2S09', 'BTrout2S03', 'BTrout1S10', 'BTrout2S08']


In [57]:
import peakutils
import scipy
from scipy.optimize import leastsq
import pandas as pd
import numpy as np
from scipy import signal


def sin_corr(tracklist_subset, tracklist, finbeat_df):
    """Computes full width at half maximum from FFT of tailbeat movement

        Args:
            tracklist_subset (list): a list of strings with the trial
                                names of the desired trials from tracklist.
                                 Note: The list (even of a single
                                 element) must be contained in square
                                 brackets.
                                 Also note: Alternatively, to iterate
                                 over all trials, set this to
                                 tracklist.keys()
            tracklist (dict): a tracklist produced by extract_data()
            finbeat_df (dict): a finbeat dataframe produced by finbeat_calc
        Returns:


"""
    sincorr_df = pd.DataFrame(columns=['Behavior', 'Pearsons', 'Pvalue',
                                       'Est.Freq', 'Est.Amplitude',
                                       'Est.Phase', 'Est.Offset'],
                              index=tracklist_subset)

    for trial in tracklist_subset:
        trialname = tracklist[trial]['sequence']
        behavior = tracklist[trial]['behavior']
        data = tracklist[trial]['data']['pt2y_smth']
        base = peakutils.baseline(data, 3)  # Find bkgrd trend
        data = data - base
        time = tracklist[trial]['data'].index.values
        periods = np.array(finbeat_df[trial]['period'])

        periods = np.sort(periods)
        periods = periods[np.logical_not(np.isnan(periods))]
        periods = np.delete(periods, [0, len(periods) - 1])
        period = np.mean(periods)
        guess_frequency = 1. / period

        amplitudes = np.array(finbeat_df[trial]['amplitude'])
        amplitudes = abs(amplitudes)
        amplitudes = np.sort(amplitudes)
        amplitudes = amplitudes[np.logical_not(np.isnan(amplitudes))]
        amplitudes = np.delete(amplitudes, [0, len(amplitudes) - 1])
        amplitude = np.mean(amplitudes) / 2
        guess_amplitude = amplitude
        guess_offset = np.mean(data)
        guess_phase = 0

        data_first_guess = guess_amplitude * np.sin(
            2 * np.pi * guess_frequency * time+ guess_phase) + guess_offset

        optimize_func = lambda x: x[0] * np.sin(
            2 * np.pi * x[1] * time + x[2]) + x[3] - data
        est_amplitude, est_frequency, est_phase, est_offset = leastsq(
            optimize_func, [guess_amplitude, guess_frequency,
                            guess_phase, guess_offset])[0]

        data_fit = est_amplitude * np.sin(
            2 * np.pi * est_frequency * time + est_phase) + est_offset
        corr = scipy.stats.pearsonr(data, data_fit)

        # print('Pearsons', corr)
        # fig = plt.figure()
        # plt.plot(time, data, '.')
        # plt.plot(time, data_fit, label='after fitting')
        # plt.legend()
        # plt.show()
        sincorr_df.loc[trialname] = pd.Series(
            {'Behavior': behavior, 'Pearsons': corr[0], 'Pvalue': corr[1],
             'Est.Freq': est_frequency, 'Est.Amplitude': est_amplitude,
             'Est.Phase': est_phase, 'Est.Offset': est_offset})
    return sincorr_df


In [61]:
corr_w_sin = sin_corr(tracklist.keys(), tracklist, finbeat_byP)
corr_w_sin.sort_values(by='Pearsons')

C:\Anaconda3\lib\site-packages\numpy\core\_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
C:\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 1000.
  warnings.warn(errors[info][0], RuntimeWarning)


,Behavior,Pearsons,Pvalue,Est.Freq,Est.Amplitude,Est.Phase,Est.Offset
BTrout2S04,B,0.232662,9.02556e-14,4.93697,-0.341453,0.141436,1.0748
BTrout2S07,B,0.347346,6.04846e-27,7.59988,0.237597,-2.3229,0.681251
BTrout1S11,B,0.431963,1.02784e-28,6.06135,0.207078,-2.31702,0.610805
BTrout2S09,B,0.483738,7.1932e-89,5.15168,0.43548,1.98011,0.971046
BTrout2S10,B,0.515603,2.8873e-89,6.72165,0.363273,1.87193,0.666519
BTrout2S03,B,0.544191,6.07113e-63,3.55012,-0.393537,-0.798182,0.629918
Bass1S13,B,0.600837,1.40268e-162,3.46761,0.752136,-1.02473,1.07259
BTrout1S10,B,0.622817,1.90662e-123,5.33826,-0.347848,0.31116,0.640047
BTrout1S08,A,0.67739,8.01924e-229,5.01042,0.489955,0.811393,0.726361
Bass1S06,B,0.702545,7.20804e-283,3.44088,-0.723482,0.0384342,0.846484


In [ ]:
import matplotlib.pyplot as plt
import pywt 
from scipy import signal

testdata = tracklist['Bass1S09']['data']['pt2y_smth']
time = tracklist['Bass1S09']['data'].index.values
framerate = float(tracklist['Bass1S09']['FPS'])
scales = np.arange(2,20,1)


wavelet_choice = pywt.Wavelet('bior3.1')
print(wavelet_choice)
for i in np.arange(2,20,1):
    freq = pywt.scale2frequency(wavelet_choice, i)
    print(': scale=',i, ' freq=', freq)

In [ ]:
for name in pywt.wavelist(kind='discrete'):
    x = time
    data = testdata
    wavelet = name
    level = 1
    order = 'freq'
    interpolation = 'nearest'
    cmap = plt.cm.cool

    wp = pywt.WaveletPacket(data, wavelet, 'symmetric', maxlevel = level)
    nodes = wp.get_level(level, order = order)
    labels = [n.path for n in nodes]
    values = np.array([n.data for n in nodes], 'd')
    values = abs(values)
    fig = plt.figure()
    fig.subplots_adjust(hspace=0.2, bottom=.03, left=.07, right=.97, top=.92)
    ax = fig.add_subplot(2, 1, 1)
    ax.set_title("Data")
    ax.plot(x, data, 'b')
    ax.set_xlim(0, x[-1])

    ax = fig.add_subplot(2, 1, 2)
    ax.set_title("Wavelet packet coefficients at level %d" % level)
    ax.imshow(values, interpolation=interpolation, cmap=cmap, aspect="auto",
              origin="lower", extent=[0, 1, 0, len(values)])
    ax.set_yticks(np.arange(0.5, len(labels) + 0.5), labels)

    # Show spectrogram and wavelet packet coefficients
    fig2 = plt.figure()
    ax2 = fig2.add_subplot(211)
    ax2.specgram(data, NFFT=64, noverlap=32, Fs=2, cmap=cmap,
                 interpolation='bilinear')
    ax2.set_title("Spectrogram of signal")
    ax3 = fig2.add_subplot(212)
    ax3.imshow(values, origin='upper', extent=[-1, 1, -1, 1],
               interpolation='nearest')
    ax3.set_title("Wavelet packet coefficients")


    plt.show()



In [ ]:

import numpy as np
import matplotlib.pyplot as plt

import pywt


x = np.linspace(0, 1, num=512)
data = np.sin(250 * np.pi * x**2)

wavelet = 'db2'
level = 4
order = "freq"  # other option is "normal"
interpolation = 'nearest'
cmap = plt.cm.cool

# Construct wavelet packet
wp = pywt.WaveletPacket(data, wavelet, 'symmetric', maxlevel=level)
nodes = wp.get_level(level, order=order)
labels = [n.path for n in nodes]
values = np.array([n.data for n in nodes], 'd')
values = abs(values)

# Show signal and wavelet packet coefficients
fig = plt.figure()
fig.subplots_adjust(hspace=0.2, bottom=.03, left=.07, right=.97, top=.92)
ax = fig.add_subplot(2, 1, 1)
ax.set_title("linchirp signal")
ax.plot(x, data, 'b')
ax.set_xlim(0, x[-1])

ax = fig.add_subplot(2, 1, 2)
ax.set_title("Wavelet packet coefficients at level %d" % level)
ax.imshow(values, interpolation=interpolation, cmap=cmap, aspect="auto",
          origin="lower", extent=[0, 1, 0, len(values)])
ax.set_yticks(np.arange(0.5, len(labels) + 0.5), labels)

# Show spectrogram and wavelet packet coefficients
fig2 = plt.figure()
ax2 = fig2.add_subplot(211)
ax2.specgram(data, NFFT=64, noverlap=32, Fs=2, cmap=cmap,
             interpolation='bilinear')
ax2.set_title("Spectrogram of signal")
ax3 = fig2.add_subplot(212)
ax3.imshow(values, origin='upper', extent=[-1, 1, -1, 1],
           interpolation='nearest')
ax3.set_title("Wavelet packet coefficients")


plt.show()